In [671]:
#Cancer Diagnosis Using Machine Learning

from sklearn.metrics import accuracy_score #To return the percent of correct predictions
import numpy as np
import pandas as pd

#reading a CSV file directly from github and assigning to a pandas DataFrame:
cancer_df = pd.read_csv("https://github.com/mpourhoma/CS4661/raw/master/Cancer.csv")
cancer_df[0::10]



,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Malignant_Cancer
0,5,1,1,1,2,1,3,1,1,0
10,5,3,3,3,2,3,4,4,1,1
20,5,4,4,9,2,10,5,6,1,1
30,9,5,8,1,2,3,2,1,5,1
40,5,3,5,5,3,3,4,10,1,1
50,5,1,3,1,2,1,2,1,1,0
60,2,2,2,1,1,1,7,1,1,0
70,1,1,1,1,2,1,3,1,1,0
80,10,3,5,1,10,5,3,10,2,1
90,1,3,1,2,2,2,5,3,2,0


In [672]:
#Creating feature matrix
feature_cols = ['Clump_Thickness','Uniformity_of_Cell_Size','Uniformity_of_Cell_Shape',
                'Marginal_Adhesion','Single_Epithelial_Cell_Size','Bare_Nuclei',
                'Bland_Chromatin','Normal_Nucleoli','Mitoses']

X = cancer_df[feature_cols]

y = cancer_df['Malignant_Cancer']

#print(X.head)
#y[::10]

In [673]:
#Splitting the dataset into testing and training sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.35, random_state=3)

# print the size of the traning set:
print(X_train.shape)
print(y_train.shape)

# print the size of the testing set:
print(X_test.shape)
print(y_test.shape)

(97, 9)
(97,)
(53, 9)
(53,)


In [674]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#instantiation of object of 'DecisionTreeClassifier' class
my_DecisionTree = DecisionTreeClassifier(random_state=3)

#Training a predictive model using the training dataset
my_DecisionTree.fit(X_train, y_train)

#Testing the predictive model (making prediction using the trained model)
y_predict_dt = my_DecisionTree.predict(X_test)

#Accuracy calculation for Decision Tree Prediction
accuracy_dt = accuracy_score(y_test, y_predict_dt)
print('Decision Tree accuracy is: ', accuracy_dt)



Decision Tree accuracy is:  0.8301886792452831


In [675]:
#Bagging ensamble learning method
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from collections import Counter

predictions = []
tally_array = []
bin_list = []
for i in range(0,19):
    #make a bootstrap sample of original "Training Dataset" with size * 0.8, then use following command to generate random bootstrap dataset
    bootstrap_size = int(0.8*len(X_train))
    x2_train, y2_train = resample(X_train, y_train, n_samples = bootstrap_size, random_state=i, replace = True)
    #Define and train a new base decision tree classifier  on this dataset in each iteration
    Base_DecisionTree = DecisionTreeClassifier(random_state=3)
    Base_DecisionTree.fit(x2_train, y2_train) #training the new predictive model
    predict_base_dt = Base_DecisionTree.predict(X_test) #making prediction using the trained model
    predictions.append(predict_base_dt) #adding the predictions to a list

#creating a new list with predictions using np in order to sum the predictions together (in order to get a sum for each column)
np_predictions = np.array(predictions) 
tally_array=np.sum(np_predictions,0)

#printing out the tally (so the number of 1's that were counted in each column)
#print(tally_array)

#creating another loop in order to create a new list that stores a '1' value for those columns where there was a majority of '1''s counted
#or a '0' for those columns in which there was a majority of '0' votes. This will give us one list
for i in range(len(tally_array)):
    if tally_array[i] > 9: #evaluate each index of tally_array list, if there was a majority of 1's in that column, make the corresponding element in the new list be a 1
        bin_list.append(1)
    else:
        bin_list.append(0)#else, then there was a majority of 0's in that index, and so the new list should add a 0 in that place. 
#print(bin_list)

#Calculating the accuracy for the list which contains the final decision on each data sample, bin_list
accuracy_vote = accuracy_score(y_test, bin_list)
print('Voting method accuracy is: ', accuracy_vote)

Voting method accuracy is:  0.9056603773584906


In [676]:
#importing and defining the classifier 
from sklearn.ensemble import RandomForestClassifier

my_RandomForest = RandomForestClassifier(n_estimators=19, bootstrap=True, random_state=3)

#training the predictive model
my_RandomForest.fit(X_train, y_train)

#Testing the predictive model
forest_predict = my_RandomForest.predict(X_test)

#calculating and printing out the accuracy
accuracy_forest = accuracy_score(y_test, forest_predict)
print('RandomForest accuracy is: ', accuracy_forest)

RandomForest accuracy is:  0.9245283018867925
